# Building AI-powered search in Amazon DocumentDB Vector Search using Amazon Bedrock and DocumentDB Vector Search
_**Using Bedrock Titan embedding model and DocumentDB `Vector Search` for similarity search on Game recommendations**_

---

---

## Contents


1. [Background](#Background)
1. [Setup](#Setup)
1. [Bedrock Model Call Preparation](#Bedrock-model-call-prepration)
1. [DocumentDB](#DocumentDB-vector-search)
1. [Evaluate Search Results](#Evaluate-DocumentDB-vector-Search-Results)

## Background

In this notebook, we'll build the core components of a textually similar Products. Often people don't know what exactly they are looking for and in that case they just type an item description and hope it will retrieve similar items.

One of the core components of searching textually similar items is a fixed length sentence/word embedding i.e. a  “feature vector” that corresponds to that text. The reference word/sentence embedding typically are generated offline and must be stored so they can be efficiently searched. In this use case we are using Amazon Bedrock Titan(https://aws.amazon.com/cn/bedrock/titan/).

To enable efficient searches for textually similar items, we'll use Amazon Bedrock Titan to generate fixed length sentence embeddings i.e “feature vectors” and use the Nearest Neighbor search in Amazon DocumentDB (with MongoDB compatibility) using the Vector Search. DocumentDB Vector Search lets you store and search for points in vector space and find the "nearest neighbors" for those points. Use cases include recommendations (for example, an "other songs you might like" feature in a music application), image recognition, and fraud detection.

Here are the steps we'll follow to build textually similar items: After some initial setup, we'll call Bedrock Titan Embedding model. Then generate feature vectors for games from the website(https://mechanicbase.com/cars/different-car-models-types) dataset. Those feature vectors will be stored in DocumentDB Vector Search vector datatype. Next, we'll explore some sample text queries, and visualize the results.

## Setup
Install required python libraries for the workshop


In [ ]:
!pip install -U pymongo  tqdm boto3 requests scikit-image

### Downloading Gaming demo data

The dataset itself consists of 29 high-resolution images, each depicting a type of car. Each of the images has five textual description. 

**Downloading Gaming Demo data**: Data originally from here: https://store.steampowered.com/charts/mostplayed

In [ ]:
import urllib.request≈
import os
import json
import boto3
from multiprocessing import cpu_count
from tqdm.contrib.concurrent import process_map
filename = 'metadata.json'

with open(filename) as json_file:
    results = json.load(json_file)
if not os.path.exists(filename):
   print ("metadata.json file not exits")
results[0]
print(results[0])

## Bedrock Model Call Preparation
prepare for Bedrock Titan model call

In [ ]:
# for bedrock model call
%pip install langchain==0.0.305 --force-reinstall

In [4]:
#for bedrock model call
import json
import os
import sys

import boto3

bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

In [5]:
# for bedrock model call
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_client)

In [40]:
# Set Up Claude Parameters
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":1,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2:1",
                    client = bedrock_client, 
                    model_kwargs = inference_modifier 
                    )

In [41]:
# calculate your LLM execution time
import time

def timer_llm(prompt, if_print=1):
    start_time = time.time()
    response = textgen_llm(prompt)
    end_time = time.time()
    elapsed_time = end_time - start_time
    if if_print == 1:
        print("----------------------------------------- OutPut -----------------------------------------")
        print("Elapsed time: ", elapsed_time, "seconds")
    return response

In [6]:
# for Bedrock Embedding model call

def generate_embeddings(data):
    r = bedrock_embeddings.embed_query(data)
    return r


## DocumentDB Vector Search

vector search for Amazon DocumentDB (with MongoDB compatibility), a new built-in capability that lets you store, index, and search millions of vectors with millisecond response times within your document database.

One of the key benefits of using pgvector is that it allows you to perform similarity searches on large datasets quickly and efficiently. This is particularly useful in industries like e-commerce, where businesses need to be able to quickly search through large product catalogs to find the items that best match a customer's preferences. It supports exact and approximate nearest neighbor search, L2 distance, inner product, and cosine distance.

To further optimize your searches, you can also use DocumentDB Vector Search's indexing features. By creating indexes on your vector data, you can speed up your searches and reduce the amount of time it takes to find the nearest neighbors to a given vector.

In this step we'll get all the translated product descriptions of *__zalandoresearch__* dataset and store those embeddings into DocumentDB Vector Search vector type.

In [47]:
# Set up a connection to your Amazon DocumentDB (MongoDB compatibility) cluster and creating the database
import pymongo

client = pymongo.MongoClient(
"docdb-vector-search-lab.cluster-cd4xami2mmof.us-east-1.docdb.amazonaws.com:27017",
username="masteruser",
password="Password1",
retryWrites=False,
tls='true',
tlsCAFile='global-bundle.pem')
db = client.similarity6
collection = db.games

In [ ]:
import pymongo
import boto3 
import json 


for x in results:
    description1 = ' '.join(x.get('descriptions', []))
    vector = generate_embeddings(description1)
    record = { "name": x.get('name'),
              "description": ' '.join(x.get('descriptions', [])),
              "recommendation":x.get('recommendation'),
          "url": x.get('url'),
          "descriptions_embeddings": vector}
    print("record",record)
    rec_id1 = collection.insert_one(record)  

collection.create_index ([("descriptions_embeddings","vector")], vectorOptions={
"lists": 1,
"similarity": "euclidean",
"dimensions": 1536}) 
client.close()

#print ("Vector embeddings has been successfully loaded into DocumentDB") 

## Evaluate DocumentDB vector Search Results

In this step we will use SageMaker realtime inference to generate embeddings for the query and use the embeddings to search the DocumentDB to retrive the nearest neighbours and retrive the relevent product images.


In [ ]:
from skimage import io
import matplotlib.pyplot as plt
import requests

p_base = '''
\n\nHuman:
你是一个游戏推荐员. 你的目标是按照<instructions>重写接下来的游戏名称，游戏类别，游戏介绍，并根据用户的主机配置以及用户是否是未成年人进行游戏推荐.

重写要求<instructions>
1. 写出游戏名称
2. 列出游戏类别
3. 优化总结游戏介绍
4. 指出玩该游戏需要的主机配置
5. 判断是否适合未成年人玩
</instructions>
参考内容为:
'''

def similarity_search(search_text):
    client = pymongo.MongoClient(
    "docdb-vector-search-lab.cluster-cd4xami2mmof.us-east-1.docdb.amazonaws.com:27017",
    username="masteruser",
    password="Password1",
    retryWrites=False,
    tls='true',
    tlsCAFile='global-bundle.pem')
    db = client.similarity6
    collection = db.games
    
    data = {"inputs": search_text}
    print(data)
    res1 = generate_embeddings(data['inputs'])
    
    query = {"vectorSearch" : {"vector" : res1, "path": "descriptions_embeddings", "similarity": "euclidean", "k": 2}}
    projection = {
    "_id":0,
    "name":1,
    "recommendation":1,
    "url":1,
    "description":1,
    "descriptions_embeddings": 1}
    r = collection.aggregate([{'$search': query},{"$project": projection}])
    
  
    urls = []
    plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    
    for x in r:
        response= timer_llm(p_base +"游戏名称:"+x["name"] + ".游戏描述:" +x["description"] +".主机配置建议："+ x["recommendation"])
        print(response)
        url = x["url"].split('?')[0]
        urldata = requests.get(url).content
        a = io.imread(url)
        plt.imshow(a)
        plt.axis('off')
        plt.show()


    client.close()

In [ ]:
Using the above function `similarity_search` , lets do some search

In [ ]:
similarity_search("多人射击游戏")